In [ ]:
! pip install langchain-community
! pip install langchain-core
! pip install langchain-google-vertexai
! pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 87.3 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.2
    Uninstalling pydantic_core-2.27.2:
      Successfully uninstalled pydantic_core-2.27.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.4
    Uninstalling pydantic-2.10.4:
      Successfully uninstalled pydantic-2.10.4
  Attempting uninstall: ht

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 7.0 MB/s eta 0:00:00


In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=''
os.environ['GEMINI_API_KEY']=''

In [ ]:
# import google.generativeai as genai

# genai.configure(api_key="")
# model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
from openai import OpenAI

def llm_generate(message):
  client = OpenAI(
    base_url="https://api-inference.huggingface.co/v1/",
    api_key=os.environ["HUGGINGFACEHUB_API_TOKEN"]
  )

  messages = message

  response = client.chat.completions.create(
      model="NousResearch/Hermes-3-Llama-3.1-8B",
    messages=messages,
    max_tokens=500,
    stream=False
  )

  return response.choices[0].message.content

In [ ]:
from sentence_transformers import SentenceTransformer
from typing import List

class MyEmbeddings:
        def __init__(self, model):
            self.model = SentenceTransformer(model, trust_remote_code=True)

        def embed_documents(self, texts: List[str]) -> List[List[float]]:
            return [self.model.encode(t).tolist() for t in texts]

        def embed_query(self, query: str) -> List[float]:
            return self.model.encode([query])[0]

embeddings=MyEmbeddings('Alibaba-NLP/gte-large-en-v1.5')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/NIPS-2017-attention-is-all-you-need-Paper.pdf")
pages = []
for page in loader.lazy_load():
   pages.append(page)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=embeddings
)
retriever = vectorstore.as_retriever()

In [ ]:
docs

[Document(metadata={'source': '/content/NIPS-2017-attention-is-all-you-need-Paper.pdf', 'page': 0}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser ∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine transla

In [ ]:
retriever.invoke({"input": "What is encoder?"})[0].page_content

'connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,\nrespectively.\n3.1 Encoder and Decoder Stacks\nEncoder: The encoder is composed of a stack of N = 6 identical layers. Each layer has two\nsub-layers. The ﬁrst is a multi-head self-attention mechanism, and the second is a simple, position-\n2'

In [ ]:
def rag_call(query):
  retrieved_docs=retriever.invoke({"input":query})
  context=[doc.page_content for doc in retrieved_docs]

  system_prompt="""
  You are a question answering bot. You will be provided context and a query, your task is to answer the query based on the context only.
  Keep your answer precise and grounded. If you dont find answer to the question respond with 'Sorry I dont know the answer.'
  """

  messages = [
      {
          "role":"system",
          "content": system_prompt
      },
      {
        "role": "user",
        "content": f"Query: {query} \n Context: {context}"
      }
    ]

  response=llm_generate(messages)
  return response

In [ ]:
def rag_call_with_history(query, chat_history=[]):
    try:
        # Retrieve relevant documents
        retrieved_docs = retriever.invoke({"input": query})
        context = [doc.page_content for doc in retrieved_docs]

        # Define the system prompt for question answering
        system_prompt = """
        You are a question answering bot. You will be provided context and a query, your task is to answer the query based on the context only.
        Keep your answer precise and grounded. If you don't find the answer to the question, respond with 'Sorry, I don't know the answer.'
        """

        # Create a list of messages with chat history and the current query
        messages = [
            {
                "role": "system",
                "content": system_prompt
            }
        ]

        # Add the last three interactions (user and assistant) from the chat history
        if len(chat_history) > 0:
            messages.insert(1, chat_history[-1])  # Most recent assistant response
        if len(chat_history) > 1:
            messages.insert(1, chat_history[-2])  # Second most recent user query
        if len(chat_history) > 2:
            messages.insert(1, chat_history[-3])  # Third most recent assistant response

        # Add the current user query along with the context
        messages.append({
            "role": "user",
            "content": f"Query: {query} \n Context: {context}"
        })

        # Generate a response from the language model
        response = llm_generate(messages)

        # Return the response
        return response

    except Exception as e:
        # Handle errors and provide a message
        return f"An error occurred: {str(e)}"


In [ ]:
rag_call("who is john cena?")

"I'm sorry, but the provided context doesn't contain any information about John Cena. The text appears to be about a technical paper related to sequence transduction models, not about any person like John Cena."

In [ ]:
def mm_rag_call(query,image_path):
  import PIL.Image
  sample_file_1 = PIL.Image.open(image_path)

  prompt = "Give summary of given image"
  summary_response = model.generate_content([prompt, sample_file_1])

  retrieved_docs=retriever.invoke({"input":str(summary_response)})
  context=[doc.page_content for doc in retrieved_docs]

  system_prompt="""
  You are a question answering bot. You will be provided context and a query,your task is to answer the query based on the context only.
  Keep your answer precise and grounded. If you dont find answer to the question respond with 'Sorry I dont know the answer.'
  """

  messages = [
      {
          "role":"system",
          "content": system_prompt
      },
      {
        "role": "user",
        "content": f"Query: {query} \n Context: {context}"
      }
    ]

  response=llm_generate(messages)
  return response

In [ ]:
mm_rag_call("Explain given architecture?",'/content/arch.png')

'The given architecture appears to be based on the Transformer model, which was introduced in the paper "Attention is All You Need" by Vaswani et al. The context provides details about the architecture of the Transformer model, specifically focusing on attention mechanisms.\n\nThe main components of the architecture described in the context are:\n\n1. Layer normalization: Applied around each sub-layer in the model.\n\n2. Scaled Dot-Product Attention: A type of attention function used in the Transformer model. It computes attention by taking the dot product of the query with all keys, dividing each by the square root of the dimension of the keys, and applying a softmax function to obtain the weights on the values.\n\n3. Multi-Head Attention: Instead of performing a single attention function, the model linearly projects the queries, keys, and values multiple times (h times) with different learned projections. This allows the model to jointly attend to information from different represent